## Exploring the Ethos dataset using Autolabel

#### Setup the API Keys for providers that you want to use

In [1]:
import os

# provide your own OpenAI API key here
os.environ['OPENAI_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxx'

#### Install the autolabel library

In [2]:
!pip install 'refuel-autolabel[openai]'

#### Download the dataset

In [2]:
from autolabel import get_data

get_data('ethos')

This downloads two datasets:
* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

### First labeling run

In [1]:
import json

from autolabel import LabelingAgent

In [2]:
# load the config
with open('config_ethos.json', 'r') as f:
     config = json.load(f)

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `attribute_extraction` (since it's an attribute extraction task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.task_guidelines`: `'You are an expert at classifying hate speech and identifying...` (how we describe the task to the LLM)
* `prompt.attributes`: `[
            {
                "name": "violence",
                ...
            },
            {
                "name": "directed_vs_generalized",
                ...
            },
            {
                "name": "gender",
                ...
            }
        ]` (the full list of labels to choose from)
* `prompt.few_shot_num`: 5 (how many labeled examples to provide to the LLM)

In [3]:
config

{'task_name': 'EthosAttributeExtraction',
 'task_type': 'attribute_extraction',
 'dataset': {'text_column': 'text', 'delimiter': ','},
 'model': {'provider': 'openai', 'name': 'gpt-3.5-turbo'},
 'prompt': {'task_guidelines': 'You are an expert at classifying hate speech and identifying the type of hate speech. Read the following tweets and extract the following attributes from the text.',
  'attributes': [{'name': 'violence',
    'options': ['not_violent', 'violent'],
    'description': 'If the tweet mentions violence towards a person or a group.'},
   {'name': 'directed_vs_generalized',
    'options': ['generalized', 'directed'],
    'description': 'If the hate speech is generalized towards a group or directed towards a specific person.'},
   {'name': 'gender',
    'options': ['true', 'false'],
    'description': 'If the hate speech uses gendered language and attacks a particular gender.'}],
  'few_shot_examples': 'seed.csv',
  'few_shot_selection': 'fixed',
  'few_shot_num': 5,
  'ex

In [4]:
# create an agent for labeling
agent = LabelingAgent(config=config)

In [5]:
# dry-run -- this tells us how much this will cost and shows an example prompt
from autolabel import AutolabelDataset
ds = AutolabelDataset("test.csv", config=config)
agent.plan(ds)

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $1.1186 │
│ Number of Examples       │ 428     │
│ Average cost per example │ $0.0026 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at classifying hate speech and identifying the type of hate speech. Read the following tweets and
extract the following attributes from the text.

You will return the extracted attributes as a json with the following keys:
{
    "violence": "If the tweet mentions violence towards a person or a group.\nOptions:\nnot_violent,violent",
    "directed_vs_generalized": "If the hate speech is generalized towards a group or directed towards a specific 
person.\nOptions:\ngeneralized,directed",
    "gender": "If the hate speech uses gendered language and attacks a particular gender.\nOptions:\ntrue,false"
}

Some examples with their output answers are provided below:

Text: You should know women's sports are a joke
Output: {"violence": "not_violent", "directed_vs_generalized": "generalized", "gender": "true"}

Text: You look like Sloth with deeper Down’s syndrome
Output: {"violence": "not_violent", "directed_vs_generalized": "directed", "gender": "false"}

Text: You look like Russian and speak like Indian. Both are disgusting go kill yourself
Output: {"violence": "violent", "directed_vs_generalized": "directed", "gender": "false"}

Text: Women deserve to be abused, I guess.
Output: {"violence": "violent", "directed_vs_generalized": "generalized", "gender": "true"}

Text: Women are made for making babies and cooking dinner and nothing else!!!
Output: {"violence": "not_violent", "directed_vs_generalized": "generalized", "gender": "true"}

Now I want you to label the following example:
Text: why haven't you faggots taken the bi-pill yet
Output:

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [8]:
# now, do the actual labeling
ds = agent.run(ds, max_items=100)

Output()

Actual Cost: 0.0665

┏━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ violence:… ┃ violence:… ┃ violence:… ┃ directed_… ┃ directed… ┃ directed_… ┃ gender:s… ┃ gender:co… ┃ gender:a… ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ 100        │ 1.0        │ 0.89       │ 100        │ 1.0       │ 0.89       │ 100       │ 1.0        │ 0.94      │
└────────────┴────────────┴────────────┴────────────┴───────────┴────────────┴───────────┴────────────┴───────────┘

We are at 89% accuracy when labeling the first 100 examples for labeling violence and gender, and 94% for gender. Let's see if we can use confidence scores to improve accuracy further by removing the less confident examples from our labeled set.

### Compute confidence scores


In [8]:
# Start computing confidence scores (using Refuel's LLMs)
os.environ['REFUEL_API_KEY'] = 'xxxxxxxxxxxxxxxxx'

In [4]:
# set `compute_confidence` -> True
config["model"]["compute_confidence"] = True

In [5]:
agent = LabelingAgent(config=config)

In [6]:
from autolabel import AutolabelDataset
ds = AutolabelDataset("test.csv", config=config)
agent.plan(ds)

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $1.1186 │
│ Number of Examples       │ 428     │
│ Average cost per example │ $0.0026 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at classifying hate speech and identifying the type of hate speech. Read the following tweets and
extract the following attributes from the text.

You will return the extracted attributes as a json with the following keys:
{
    "violence": "If the tweet mentions violence towards a person or a group.\nOptions:\nnot_violent,violent",
    "directed_vs_generalized": "If the hate speech is generalized towards a group or directed towards a specific 
person.\nOptions:\ngeneralized,directed",
    "gender": "If the hate speech uses gendered language and attacks a particular gender.\nOptions:\ntrue,false"
}

Some examples with their output answers are provided below:

Text: You should know women's sports are a joke
Output: {"violence": "not_violent", "directed_vs_generalized": "generalized", "gender": "true"}

Text: You look like Sloth with deeper Down’s syndrome
Output: {"violence": "not_violent", "directed_vs_generalized": "directed", "gender": "false"}

Text: You look like Russian and speak like Indian. Both are disgusting go kill yourself
Output: {"violence": "violent", "directed_vs_generalized": "directed", "gender": "false"}

Text: Women deserve to be abused, I guess.
Output: {"violence": "violent", "directed_vs_generalized": "generalized", "gender": "true"}

Text: Women are made for making babies and cooking dinner and nothing else!!!
Output: {"violence": "not_violent", "directed_vs_generalized": "generalized", "gender": "true"}

Now I want you to label the following example:
Text: why haven't you faggots taken the bi-pill yet
Output:

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [7]:
ds = agent.run(ds, max_items=100)

Output()

Actual Cost: 0.0

┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓
┃ viole… ┃ violen… ┃ viole… ┃ violen… ┃ direc… ┃ direct… ┃ direc… ┃ direct… ┃ gende… ┃ gender… ┃ gende… ┃ gender… ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩
│ 100    │ 1.0     │ 0.89   │ 0.7171  │ 100    │ 1.0     │ 0.89   │ 0.6885  │ 100    │ 1.0     │ 0.94   │ 0.8652  │
└────────┴─────────┴────────┴─────────┴────────┴─────────┴────────┴─────────┴────────┴─────────┴────────┴─────────┘

In [13]:
ds.df

,Unnamed: 0,text,violence,directed_vs_generalized,gender,EthosAttributeExtraction_label,violence_label,directed_vs_generalized_label,gender_label,EthosAttributeExtraction_error,EthosAttributeExtraction_prompt,EthosAttributeExtraction_successfully_labeled,EthosAttributeExtraction_annotation,EthosAttributeExtraction_confidence,violence_confidence,directed_vs_generalized_confidence,gender_confidence
0,5,why haven't you faggots taken the bi-pill yet,violent,directed,false,"{'violence': 'not_violent', 'directed_vs_gener...",not_violent,directed,false,None,You are an expert at classifying hate speech a...,True,successfully_labeled=True label={'violence': '...,"{'violence': 0.7650375365, 'directed_vs_genera...",0.765038,0.603027,0.836589
1,6,Why don’t you go beck your god to murder you? ...,violent,directed,false,"{'violence': 'not_violent', 'directed_vs_gener...",not_violent,directed,false,None,You are an expert at classifying hate speech a...,True,successfully_labeled=True label={'violence': '...,"{'violence': 0.75377655025, 'directed_vs_gener...",0.753777,0.584473,0.925618
2,7,why does that african american monkey hold gun...,not_violent,directed,false,"{'violence': 'not_violent', 'directed_vs_gener...",not_violent,directed,false,None,You are an expert at classifying hate speech a...,True,successfully_labeled=True label={'violence': '...,"{'violence': 0.79421997, 'directed_vs_generali...",0.794220,0.399414,0.898600
3,8,Why does Bulgaria exist as a country? It's one...,not_violent,directed,false,"{'violence': 'not_violent', 'directed_vs_gener...",not_violent,generalized,false,None,You are an expert at classifying hate speech a...,True,successfully_labeled=True label={'violence': '...,"{'violence': 0.825805665, 'directed_vs_general...",0.825806,0.873291,0.863770
4,9,Why do negroes tear down their own neighborhoo...,not_violent,generalized,false,"{'violence': 'not_violent', 'directed_vs_gener...",not_violent,generalized,false,None,You are an expert at classifying hate speech a...,True,successfully_labeled=True label={'violence': '...,"{'violence': 0.7605590775, 'directed_vs_genera...",0.760559,0.715820,0.897949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,100,If I had Down syndrome I would kill myself,violent,generalized,false,"{'violence': 'not_violent', 'directed_vs_gener...",not_violent,generalized,false,None,You are an expert at classifying hate speech a...,True,successfully_labeled=True label={'violence': '...,"{'violence': 0.7584991455, 'directed_vs_genera...",0.758499,0.510834,0.953288
96,101,I’m searching for a hitman right now to kill t...,violent,directed,true,"{'violence': 'violent', 'directed_vs_generaliz...",violent,directed,true,None,You are an expert at classifying hate speech a...,True,successfully_labeled=True label={'violence': '...,"{'violence': 0.9345703000000001, 'directed_vs_...",0.934570,0.791016,0.725342
97,102,I remember once I was in charge of a disabled ...,violent,directed,false,"{'violence': 'violent', 'directed_vs_generaliz...",violent,directed,false,None,You are an expert at classifying hate speech a...,True,successfully_labeled=True label={'violence': '...,"{'violence': 0.9240722699999999, 'directed_vs_...",0.924072,0.851562,0.916016
98,103,I just don't like Homosexuals raising kids it'...,not_violent,generalized,false,"{'violence': 'not_violent', 'directed_vs_gener...",not_violent,generalized,false,None,You are an expert at classifying hate speech a...,True,successfully_labeled=True label={'violence': '...,"{'violence': 0.821228025, 'directed_vs_general...",0.821228,0.780518,0.837240
